# Sigma_density: M Dwarfs with Spectroscopic Constraints

Calculating density constraints for M Dwarf KOIs with spectroscopic constraints on properties.

1. From spectroscopy, we know Temperature, Mass, and Radius.
2. Which isochrones (stellar models, with Temperature, Mass, and Radius as parameters) fit above star?
3. Now take into account GAIA distance for the star. From this, we know the Luminosity.
4. Look back at stellar models - which isochrone fits the Temperature, Mass Radius, and Luminority (from GAIA)?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from pprint import pprint
import pandas as pd
from tqdm import tqdm
from astropy.table import Table

plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
headers = ['KOI', 'KIC', 'Teff', 'ETeff', 'eTeff', 'Fe/H', 'e_Fe/H', 'M/H', 'e_M/H', 'SpType', 'Mstar', 'e_Mstar', 'Rstar', 'e_Rstar', 'Dis', 'Fe/H-T', 'e_Fe/H-T', 'Fe/H-M', 'e_Fe/H-M', 'M/H-M', 'e_M/H-M']

For now, removed any rows with missing data and saved them into "muirhead_data_no_missing_data.txt".

In [3]:
muirhead_data = pd.read_csv("muirhead_data_no_missing_data.txt", sep=" ")
#muirhead_data

In [4]:
isochrones = pd.read_csv('isochrones_sdss_spitzer_lowmass.dat', sep='\s\s+', engine='python')
#isochrones

#### First, let's work with muirhead_data[0]: KIC 5868793.

1. Take Teff, Mass, and Radius.
2. Which isochrones fit these Teff, Mass, and Radius?

#### Reading Gaia-Kepler data (luminosities)

In [5]:
dat = Table.read('kepler_dr2_1arcsec.fits', format='fits')
df = dat.to_pandas()

/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_result_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'r_modality_flag' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err1' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/sheilasagear/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'teff_err2' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
muirhead_gaia = df[df['kepid'].isin(list(muirhead_data.KIC))]
muirhead_gaia = muirhead_gaia.reset_index()
muirhead_gaia.rename(columns={"index": "KIC"})

#muirhead_comb: planet hosts with spectroscopic data + Gaia/Kepler data in one table
muirhead_comb = pd.concat([muirhead_data, muirhead_gaia], axis=1)

In [17]:
def fit_isochrone(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia data for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ONLY ON SPECTROSCOPY.
       """            
    
    iso_fits_final = list()
    
    #test each star in spectroscopy sample:
    #for i in tqdm(range(len(muirhead_comb))):
    for i in tqdm(range(1)):

        iso_fits = pd.DataFrame() 

        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]

        #test each stellar model to see if it falls within error bars:
        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])

        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]

        iso_fits_final.append(iso_fits)

    return iso_fits_final


In [18]:
isos = fit_isochrone(muirhead_comb, isochrones)

100%|██████████| 1/1 [01:30<00:00, 90.56s/it]


In [9]:
#for i in range(len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str([i]) + ".csv")

In [10]:
#for i in range(12, len(isos)):
#    isos[i].to_csv("isochrone_fits/spectroscopy/iso_fits_" + str(isos[i]['KIC'].iloc[0]) + ".csv")

In [19]:
def fit_isochrone_lum(data, isochrones):
    """
       Inputs:
       data: pd.DataFrame. Spectroscopic data + Kepler/Gaia for n stars in one table. (muirhead_comb)
       isochrones: pd.DataFrame. Isochrones table. (isochrones)
       
       Returns: list of pd.DataFrames. Each element of list is a pd.DataFrame of the isochrones that fit this star (index) BASED ON SPECTROSCOPY AND GAIA LUMINOSITY.
       """    
    
    iso_fits_final = list()
    
    #for i in tqdm(range(len(muirhead_comb))):
    for i in range(1):
        
        iso_fits = pd.DataFrame() 
    
        Teff_range = [data.Teff[i]-data.eTeff[i], data.Teff[i]+data.ETeff[i]]
        Mstar_range = [data.Mstar[i]-data.e_Mstar[i], data.Mstar[i]+data.e_Mstar[i]]
        Rstar_range = [data.Rstar[i]-data.e_Rstar[i], data.Rstar[i]+data.e_Rstar[i]]
        lum_range = [data.lum_val[i]-data.lum_percentile_lower[i], data.lum_val[i]+data.lum_percentile_lower[i]]

        for j in range(len(isochrones)):
            if Teff_range[0] < 10**isochrones.logt[j] < Teff_range[1] and Mstar_range[0] < isochrones.mstar[j] < Mstar_range[1] and Rstar_range[0] < isochrones.radius[j] < Rstar_range[1] and lum_range[0] < 10**isochrones.logl_ls[j] < lum_range[1]:
                iso_fits = iso_fits.append(isochrones.loc[[j]])
        
        iso_fits['KIC'] = muirhead_comb['KIC'][i]
        iso_fits['KOI'] = muirhead_comb['KOI'][i]
        
        iso_fits_final.append(iso_fits)
            
    return iso_fits_final

In [20]:
iso_lums = fit_isochrone_lum(muirhead_comb, isochrones)


In [14]:
#for i in range(37, len(iso_lums)):
#    try:
#        iso_lums[i].to_csv("isochrone_fits/spectroscopy_luminosity/iso_lums_" + str(iso_lums[i]['KIC'].iloc[0]) + ".csv")
#    except IndexError:
#        pass

### Calculate rho

In [16]:
#Let's just do 1 star for now
ntargs = 1

In [1]:
def find_density_dist_symmetric(ntargs, masses, masserr1, masserr2, radii, raderr1, raderr2):
    
    rho_dist = np.zeros((ntargs, 1000))
    mass_dist = np.zeros((ntargs, 1000))
    rad_dist = np.zeros((ntargs, 1000))

    #star: indexing star
    #point: indexing PDF point for star
    
    for star in tqdm(range(ntargs)):
        
        rho_temp = np.zeros(1200)
        mass_temp = np.zeros(1200)
        rad_temp = np.zeros(1200)
        
        #####
        mass_temp = np.random.normal(masses[star], np.mean((abs(masserr2[star]), abs(masserr1[star]))), 1200)
        #len 1200
        rad_temp = np.random.normal(radii[star], np.mean((abs(raderr2[star]), abs(raderr1[star]))), 1200)
        #len 1200
        #####
        
        #for j from 0 to 1200
        #for each point in individual star PDF
        #Adding each density point to rho_temp (specific to this star)
        for point in range(len(mass_temp)):
            #if mass_dist[point] >= 0. and rad_dist[point] >= 0:
            if True:
                rho_temp[point] = density(mass_temp[point], rad_temp[point], sol_density)
                
        #Now rho_temp is a n-long array with this star. We want it to be 1000-long exactly
                                 
        while len(rho_temp) > 1000:
            temp_ind = np.random.randint(0, len(rho_temp)-1)
            rho_temp = np.delete(rho_temp, temp_ind)
            mass_temp = np.delete(mass_temp, temp_ind)
            rad_temp = np.delete(rad_temp, temp_ind)
        else:
            rho_dist[star] = rho_temp
            mass_dist[star] = mass_temp
            rad_dist[star] = rad_temp
    
    return rho_dist, mass_dist, rad_dist

In [3]:
def density(mass, radius, sol_density):
    """Mass in solar density
    Radius in solar density
    sol_density in kg/m^3"""

    return ((mass*1.989e30)/((4./3.)*np.pi*radius**3*696.34e6**3))/sol_density

In [ ]:
rho, mass, radius = find_density_dist_symmetric(ntargs, mass_dwarf, masserr1_dwarf, masserr2_dwarf, rad_dwarf, raderr1_dwarf, raderr2_dwarf, logg_dwarf)